# Experiment README

## Table of Contents

* [Overview of Experiment Architecture](#Overview-of-Experiment-Architecture)
* [Experiment Workflow](#Experiment-Workflow)
    * [Modifying State Variables](#Modifying-State-Variables)
    * [Modifying System Parameters](#Modifying-System-Parameters)
    * [Executing Experiments](#Executing-Experiments)
    * [Post-processing and Analysing Results](#Post-processing-and-Analysing-Results)
    * [Visualizing Results](#Visualizing-Results)
* [Creating New, Customized Experiment Notebooks](#Creating-New,-Customized-Experiment-Notebooks)
    * Step 1: Select an experiment template
    * Step 2: Create a new notebook
    * Step 3: Customize the experiment
    * Step 4: Execute the experiment
* [Advanced Experiment-configuration & Simulation Techniques](#Advanced-Experiment-configuration-&-Simulation-Techniques)
    * [Setting Simulation Timesteps and Unit of Time `dt`](#Setting-Simulation-Timesteps-and-Unit-of-Time-dt)
    * [Changing the Ethereum Network Upgrade Stage](#Changing-the-Ethereum-Network-Upgrade-Stage)
    * [Performing Large-scale Experiments](#Performing-Large-scale-Experiments)

# Overview of Experiment Architecture

The experiment architecture is composed of the following four elements – the **model**, **default experiment**, **experiment templates**, and **experiment notebooks**:

1. The **model** is initialized with a default Initial State and set of System Parameters defined in the `model` module.
2. The **default experiment** – in the `experiments.default_experiment` module – is an experiment composed of a single simulation that uses the default cadCAD **model** Initial State and System Parameters. Additional default simulation execution settings such as the number of timesteps and runs are also set in the **default experiment**.
3. The **experiment templates** – in the `experiments.templates` module – contain pre-configured analyses based on the **default experiment**. Examples include `experiments.templates.time_domain_analysis` (simulation in the time-domain over a period of 5 years) and `experiments.templates.eth_price_sweep_analysis` (simulation in the phase-space sweeping over discrete ETH Price values).
4. The **experiment notebooks** perform various scenario analyses by importing existing **experiment templates**, optionally modifying the Initial State and System Parameters within the notebook, and then executing them.

# Experiment Workflow

If you just want to run (execute) existing experiment notebooks, simply open the respective notebook and execute all cells.

Depending on the chosen template and planned analysis, the required imports might differ slightly from the below standard dependencies:

In [1]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
from pprint import pprint

# Project dependencies
import model.constants as constants
import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

time: 6.47 s (started: 2023-03-18 18:47:49 +05:00)


We can then import the default experiment, and create a copy of the simulation object – we create a new copy for each analysis we'd like to perform:

In [2]:
import experiments.default_experiment as default_experiment
import experiments.templates.time_domain_analysis as time_domain_analysis
import experiments.templates.eth_price_eth_staked_grid_analysis as eth_price_eth_staked_grid_analysis

simulation_analysis_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_analysis_2 = copy.deepcopy(time_domain_analysis.experiment.simulations[0])
simulation_analysis_3 = copy.deepcopy(eth_price_eth_staked_grid_analysis.experiment.simulations[0])

time: 5.2 s (started: 2023-03-18 18:47:56 +05:00)


We can use the `display_code` method to see the configuration of the default experiment before making changes:

In [3]:
display_code(default_experiment)

"""
The default experiment with default model Initial State, System Parameters, and Simulation Configuration.

The defaults are defined in their respective modules:
* Initial State in `model/state_variables.py`
* System Parameters in `model/system_parameters.py`
* Simulation Configuration in `experiments/simulation_configuration.py`
"""

from radcad import Simulation, Experiment, Backend

from model import model
from experiments.simulation_configuration import TIMESTEPS, DELTA_TIME, MONTE_CARLO_RUNS


# Create Model Simulation
simulation = Simulation(
    model=model,
    timesteps=TIMESTEPS,
    runs=MONTE_CARLO_RUNS
)
# Create Experiment of single Simulation
experiment = Experiment([simulation])
# Configure Simulation & Experiment engine
simulation.engine = experiment.engine
experiment.engine.backend = Backend.SINGLE_PROCESS
experiment.engine.deepcopy = False
experiment.engine.drop_substeps = True

time: 266 ms (started: 2023-03-18 18:48:01 +05:00)


## Modifying State Variables

To view what the Initial State (radCAD model-configuration setting `initial_state`) of the State Variables are, and to what value they have been set, we can inspect the dictionary as follows:

In [4]:
pprint(simulation_analysis_1.model.initial_state)

{'amount_slashed': 0,
 'attestation_penalties': 0,
 'average_effective_balance': 32000000000.0,
 'base_fee_per_gas': 1,
 'base_reward': 0,
 'block_proposer_reward': 0,
 'eth_price': 1251.4771311475413,
 'eth_staked': 18749149.19859209,
 'eth_supply': 122373866.2178,
 'head_reward': 0,
 'network_issuance': 0,
 'number_of_active_validators': 551710,
 'number_of_awake_validators': 551710,
 'number_of_validators_in_activation_queue': 0,
 'pow_issuance': 0,
 'source_reward': 0,
 'stage': None,
 'supply_inflation': 0,
 'sync_committee_penalties': 0,
 'sync_reward': 0,
 'target_reward': 0,
 'timestamp': None,
 'total_base_fee': 0,
 'total_network_costs': 0,
 'total_online_validator_rewards': 0,
 'total_priority_fee_to_miners': 0,
 'total_priority_fee_to_validators': 0,
 'total_profit': 0,
 'total_profit_yields': 0,
 'total_realized_mev_to_miners': 0,
 'total_realized_mev_to_validators': 0,
 'total_revenue': 0,
 'total_revenue_yields': 0,
 'validating_penalties': 0,
 'validating_rewards': 0,
 

To modify the value of **State Variables** for a specific analysis, you need to select the relevant simulation and update the chosen model Initial State. For example, updating the `eth_supply` Initial State to `100e6` (100 million ETH):

In [5]:
simulation_analysis_1.model.initial_state.update({
    "eth_supply": 100e6, 
})

time: 109 ms (started: 2023-03-18 18:48:02 +05:00)


## Modifying System Parameters

To view what the System Parameters (radCAD model configuration setting `params`) are, and to what value they have been set, we can inspect the dictionary as follows:

In [6]:
pprint(simulation_analysis_1.model.params)

{'BASE_FEE_MAX_CHANGE_DENOMINATOR': [8],
 'BASE_REWARD_FACTOR': [64],
 'CHURN_LIMIT_QUOTIENT': [65536],
 'EFFECTIVE_BALANCE_INCREMENT': [1000000000.0],
 'ELASTICITY_MULTIPLIER': [2],
 'MAX_EFFECTIVE_BALANCE': [32000000000.0],
 'MAX_VALIDATOR_COUNT': [None],
 'MIN_PER_EPOCH_CHURN_LIMIT': [4],
 'MIN_SLASHING_PENALTY_QUOTIENT': [64],
 'PROPORTIONAL_SLASHING_MULTIPLIER': [2],
 'PROPOSER_REWARD_QUOTIENT': [8],
 'PROPOSER_WEIGHT': [8],
 'SYNC_REWARD_WEIGHT': [2],
 'TIMELY_HEAD_WEIGHT': [14],
 'TIMELY_SOURCE_WEIGHT': [14],
 'TIMELY_TARGET_WEIGHT': [26],
 'WEIGHT_DENOMINATOR': [64],
 'WHISTLEBLOWER_REWARD_QUOTIENT': [512],
 'base_fee_process': [<function Parameters.<lambda> at 0x00000223D77EFC10>],
 'daily_pow_issuance': [13527.628415300547],
 'date_eip1559': [datetime.datetime(2021, 8, 4, 0, 0)],
 'date_pos': [datetime.datetime(2022, 3, 1, 0, 0)],
 'date_start': [datetime.datetime(2023, 3, 18, 18, 47, 55, 127169)],
 'dt': [225],
 'eth_price_process': [<function Parameters.<lambda> at 0x000002

To modify the value of **System Parameters** for a specific analysis, you need to select the relevant simulation, and update the chosen model System Parameter (which is a list of values). For example, updating the `BASE_REWARD_FACTOR` System Parameter to a sweep of two values, `64` and `32`:

In [7]:
simulation_analysis_1.model.params.update({
    "BASE_REWARD_FACTOR": [64, 32],
})

time: 109 ms (started: 2023-03-18 18:48:02 +05:00)


## Executing Experiments

We can now execute our custom analysis and retrieve the post-processed Pandas DataFrame using the `run(...)` method:

In [8]:
df, exceptions = run(simulation_analysis_1)

2023-03-18 18:48:02,879 - root - INFO - Running experiment
2023-03-18 18:48:02,885 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-03-18 18:48:02,999 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-03-18 18:48:03,067 - root - INFO - Experiment complete in 0.1870713233947754 seconds
2023-03-18 18:48:03,067 - root - INFO - Post-processing results
2023-03-18 18:48:03,448 - root - INFO - Post-processing complete in 0.3811314105987549 seconds
time: 671 ms (started: 2023-03-18 18:48:02 +05:00)


## Post-processing and Analysing Results

We can see that we had no exceptions for the single simulation we executed:

In [9]:
exceptions[0]['exception'] == None

True

time: 125 ms (started: 2023-03-18 18:48:03 +05:00)


We can simply display the Pandas DataFrame to inspect the results. This DataFrame already has some default post-processing applied (see [experiments/post_processing.py](../post_processing.py))

In [10]:
df

,stage,timestamp,eth_price,eth_supply,eth_staked,supply_inflation,network_issuance,pow_issuance,number_of_validators_in_activation_queue,average_effective_balance,...,target_reward_eth,head_reward_eth,block_proposer_reward_eth,sync_reward_eth,whistleblower_rewards_eth,amount_slashed_eth,daily_revenue_yields_pct,cumulative_revenue_yields_pct,daily_profit_yields_pct,cumulative_profit_yields_pct
1,4.0,2023-03-18 18:47:55.127169,1251.477131,9.999857e+07,17676320.0,-0.005225,-1430.453509,0,0,3.200000e+10,...,746.388561,401.901533,234.344917,58.586229,0.014063,0.1125,0.011460,0.011460,0.010495,0.010495
2,4.0,2023-03-19 18:47:55.127169,1251.477131,9.999714e+07,17697920.0,-0.005217,-1428.242171,0,0,3.200000e+10,...,747.300628,402.392646,234.631281,58.657820,0.014063,0.1125,0.011458,0.022918,0.010494,0.020989
3,4.0,2023-03-20 18:47:55.127169,1251.477131,9.999571e+07,17719520.0,-0.005222,-1429.802282,0,0,3.200000e+10,...,746.657160,402.046163,234.429250,58.607312,0.014063,0.1125,0.011435,0.034353,0.010472,0.031461
4,4.0,2023-03-21 18:47:55.127169,1251.477131,9.999428e+07,17741120.0,-0.005214,-1427.595541,0,0,3.200000e+10,...,747.567331,402.536255,234.715018,58.678754,0.014063,0.1125,0.011434,0.045787,0.010471,0.041932
5,4.0,2023-03-22 18:47:55.127169,1251.477131,9.999286e+07,17762720.0,-0.005206,-1425.388800,0,0,3.200000e+10,...,748.477502,403.026347,235.000786,58.750196,0.014063,0.1125,0.011433,0.057220,0.010469,0.052402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,4.0,2024-03-07 18:47:55.127169,1251.477131,9.920085e+07,25344320.0,-0.007937,-2155.839179,0,0,3.200000e+10,...,447.203037,240.801635,140.409117,35.102279,0.014063,0.1125,0.005130,2.017531,0.004508,1.785516
718,4.0,2024-03-08 18:47:55.127169,1251.477131,9.919869e+07,25365920.0,-0.007954,-2160.314028,0,0,3.200000e+10,...,445.357384,239.807822,139.829634,34.957408,0.014063,0.1125,0.005108,2.022639,0.004487,1.790003
719,4.0,2024-03-09 18:47:55.127169,1251.477131,9.919653e+07,25387520.0,-0.007951,-2159.394553,0,0,3.200000e+10,...,445.736622,240.012027,139.948704,34.987176,0.014063,0.1125,0.005108,2.027747,0.004486,1.794489
720,4.0,2024-03-10 18:47:55.127169,1251.477131,9.919437e+07,25409120.0,-0.007948,-2158.475077,0,0,3.200000e+10,...,446.115860,240.216232,140.067774,35.016943,0.014063,0.1125,0.005107,2.032854,0.004486,1.798974


time: 157 ms (started: 2023-03-18 18:48:03 +05:00)


We can also use Pandas for numerical analyses:

In [11]:
# Get the maximum validating rewards in ETH for each subset
df.groupby('subset')['validating_rewards'].max() / constants.gwei

subset
0    2210.651904
1    1104.317704
Name: validating_rewards, dtype: float64

time: 125 ms (started: 2023-03-18 18:48:03 +05:00)


## Visualizing Results

Once we have the results post-processed and in a Pandas DataFrame, we can use Plotly for plotting our results:

In [12]:
# Plot the total validating rewards in ETH for each subset
px.line(df, x='timestamp', y='validating_rewards_eth', facet_col='subset')

time: 812 ms (started: 2023-03-18 18:48:04 +05:00)


In [13]:
# Plot the individual validating rewards in ETH for each subset
visualizations.plot_validating_rewards(df, subplot_titles=["Base Reward Factor = 64", "Base Reward Factor = 32"])

time: 281 ms (started: 2023-03-18 18:48:04 +05:00)


# Creating New, Customized Experiment Notebooks

If you want to create an entirely new analysis, you'll need to create a new experiment notebook, which entails the following steps:
* Step 1: Select an experiment template from the `experiments/templates/` directory to start from. If you'd like to create your own template, the [example_analysis.py](../templates/example_analysis.py) template gives an example of extending the default experiment to override default State Variables and System Parameters that you can copy.
* Step 2: Create a new notebook in the `experiments/notebooks/` directory, using the [template.ipynb](./template.ipynb) notebook as a guide, and import the experiment from the experiment template.
* Step 3: Customize the experiment for your specific analysis.
* Step 4: Execute your experiment, post-process and analyze the results, and create Plotly charts!

# Advanced Experiment-configuration & Simulation Techniques

## Setting Simulation Timesteps and Unit of Time `dt`

In [14]:
from experiments.simulation_configuration import TIMESTEPS, DELTA_TIME, SIMULATION_TIME_MONTHS

time: 125 ms (started: 2023-03-18 18:48:05 +05:00)


We can configure the number of simulation timesteps `TIMESTEPS` from a simulation time in months `SIMULATION_TIME_MONTHS`, multiplied by the number of epochs in a month, and divided by the simulation unit of time `DELTA_TIME`:

In [15]:
SIMULATION_TIME_MONTHS / 12  # Divide months by 12 to get number of years

1.0

time: 109 ms (started: 2023-03-18 18:48:05 +05:00)


`DELTA_TIME` is a variable that sets how many epochs are simulated for each timestep. Sometimes, if we don't need a finer granularity (1 epoch per timestep, for example), we can then set `DELTA_TIME` to a larger value for better performance. The default value is 1 day or `225` epochs. This means that all our time-based states will be for a period of 1 day (we call this "aggregation"), which is convenient.

In [16]:
DELTA_TIME

225

time: 110 ms (started: 2023-03-18 18:48:05 +05:00)


`TIMESTEPS` is now simply the simulation time in months, multiplied by the number of epochs in a month, divided by `DELTA_TIME`:

```python
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME
```

In [17]:
TIMESTEPS

360

time: 110 ms (started: 2023-03-18 18:48:05 +05:00)


Finally, to set the simulation timesteps (note, you may have to update the environmental processes that depend on the number of timesteps, and override the relevant parameters):

In [18]:
simulation_analysis_1.timesteps = TIMESTEPS

time: 140 ms (started: 2023-03-18 18:48:06 +05:00)


### Considerations When Performing Efficient Phase-space Simulations

In `simulation_analysis_3`, `timesteps` is decreased to `1`, but `dt` is increased to `TIMESTEPS * DELTA_TIME`, where `DELTA_TIME` is the full duration of the simulation. This produces the final result in a single processing cycle, producing the full phase-space with very low processing overhead. This is achieved by ignoring all time-series information between the beginning and end of the simulation.

There is a test function `test_dt(...)` in `tests/test_integration.py` that can be used to verify that no information is lost due to the approximations taken along the time axis for the specific State Variables that you are interested in, and that your custom code has not introduced mechanisms that might not work well with this kind of approximation. 

An example of a type of mechanism that would not work with this kind of approximation is a mechanism that implements some form of feedback loop.

## Changing the Ethereum Network Upgrade Stage

The model operates over different Ethereum-network upgrade stages. The default experiment operates in the "post-merge" Proof of Stake stage.

`Stage` is an Enum; we can import it and see what options we have:

In [19]:
from model.types import Stage

time: 110 ms (started: 2023-03-18 18:48:06 +05:00)


The model is well documented, and we can view the Python docstring to see what a Stage is, and create a dictionary to view the Enum members:

In [20]:
print(Stage.__doc__)
{e.name: e.value for e in Stage}

Stages of the Ethereum network upgrade process finite-state machine


{'ALL': 1, 'BEACON_CHAIN': 2, 'EIP1559': 3, 'PROOF_OF_STAKE': 4}

time: 125 ms (started: 2023-03-18 18:48:06 +05:00)


The `PROOF_OF_STAKE` stage, for example, assumes the Beacon Chain has been implemented, EIP-1559 has been enabled, and POW issuance is disabled:

In [21]:
display_code(Stage)

class Stage(Enum):
    """Stages of the Ethereum network upgrade process finite-state machine"""

    ALL = 1
    """Transition through all stages"""
    BEACON_CHAIN = 2
    """Beacon Chain implemented; EIP1559 disabled; POW issuance enabled"""
    EIP1559 = 3
    """Beacon Chain implemented; EIP1559 enabled; POW issuance enabled"""
    PROOF_OF_STAKE = 4
    """Beacon Chain implemented; EIP1559 enabled; POW issuance disabled"""

time: 109 ms (started: 2023-03-18 18:48:06 +05:00)


As before, we can update the "stage" System Parameter to set the relevant Stage:

In [22]:
simulation_analysis_1.model.params.update({
    "stage": [Stage.PROOF_OF_STAKE]
})

time: 125 ms (started: 2023-03-18 18:48:06 +05:00)


## Performing Large-scale Experiments

When executing an experiment, we have three degrees of freedom - **simulations, runs, and subsets** (parameter sweeps).

We can have multiple simulations for a single experiment, multiple runs for every simulation, and we can have multiple subsets for every run. Remember that `simulation`, `run`, and `subset` are simply additional State Variables set by the radCAD engine during execution – we then use those State Variables to index the results for a specific dimension, e.g. simulation 1, run 5, and subset 2.

Each dimension has a generally accepted purpose:
* Simulations are used for A/B testing
* Runs are used for Monte Carlo analysis
* Subsets are used for parameter sweeps

In some cases, we break these "rules" to allow for more degrees of freedom or easier configuration.

One example of this is the `eth_price_eth_staked_grid_analysis` experiment template we imported earlier:

In [23]:
display_code(eth_price_eth_staked_grid_analysis)

"""
# ETH Price / ETH Staked Grid Analysis

Creates a cartesian product grid of ETH price and ETH staked processes, for phase-space analyses.
"""

import numpy as np
import copy
from radcad.utils import generate_cartesian_product_parameter_sweep

from model.state_variables import eth_staked, eth_supply, eth_price_max
from experiments.default_experiment import experiment, TIMESTEPS, DELTA_TIME


# Make a copy of the default experiment to avoid mutation
experiment = copy.deepcopy(experiment)

sweep = generate_cartesian_product_parameter_sweep({
    # ETH price range from 100 USD/ETH to the maximum over the last 12 months
    "eth_price_samples": np.linspace(start=100, stop=eth_price_max, num=20),
    # ETH staked range from current ETH staked to minimum of 2 x ETH staked and 30% of total ETH supply
    "eth_staked_samples": np.linspace(start=eth_staked, stop=min(eth_staked * 2, eth_supply * 0.3), num=20),
})

parameter_overrides = {
    "eth_price_process": [
        lambda run, _timestep: sweep["eth_price_samples"][run - 1]
    ],
    "eth_staked_process": [
        lambda run, _timestep: sweep["eth_staked_samples"][run - 1]
    ]
}

# Override default experiment parameters
experiment.simulations[0].model.params.update(parameter_overrides)
# Set runs to number of combinations in sweep
experiment.simulations[0].runs = len(sweep["eth_price_samples"])
# Run single timestep, set unit of time to multiple epochs
experiment.simulations[0].timesteps = 1
experiment.simulations[0].model.params.update({"dt": [TIMESTEPS * DELTA_TIME]})

time: 141 ms (started: 2023-03-18 18:48:06 +05:00)


Here, we create a grid of two State Variables – ETH price and ETH staked – using the `eth_price_process` and `eth_staked_process`.

Instead of sweeping the two System Parameters to create different subsets, we pre-generate all possible combinations of the two values first and use the specific `run` to index the data, i.e. for each run we get a new ETH price and ETH staked sample.

This allows the experimenter (you!) to use a parameter sweep on top of this analysis if they choose, and we have kept one degree of freedom.

### Composing an Experiment Using **simulations, runs, and subsets**

In [24]:
from radcad import Experiment, Engine, Backend


# Create a new Experiment of three Simulations:
# * Simulation Analysis 1 has one run and two subsets – a parameter sweep of two values (BASE_REWARD_FACTOR = [64, 32])
# * Simulation Analysis 2 has one run and one subset – a basic simulation configuration
# * Simulation Analysis 3 has 400 runs (20 * 20) and one subset – a parameter grid indexed using `run`
experiment = Experiment([simulation_analysis_1, simulation_analysis_2, simulation_analysis_3])

time: 109 ms (started: 2023-03-18 18:48:07 +05:00)


### Configuring the radCAD Engine for High Performance

To improve simulation performance for large-scale experiments, we can set the following settings using the radCAD `Engine`. Both Experiments and Simulations have the same `Engine`; when executing an `Experiment` we set these settings on the `Experiment` instance:

In [25]:
# Configure Experiment Engine
experiment.engine = Engine(
    # Use a single process; the overhead of creating multiple processes
    # for parallel-processing is only worthwhile when the Simulation runtime is long
    backend = Backend.SINGLE_PROCESS,
    # Disable System Parameter and State Variable deepcopy:
    # * Deepcopy prevents mutation of state at the cost of lower performance
    # * Disabling it leaves it up to the experimenter to use Python best-practises to avoid 
    # state mutation, like manually using `copy` and `deepcopy` methods before
    # performing mutating calculations when necessary
    deepcopy = False,
    # If we don't need the state history from individual substeps,
    # we can get rid of them for higher performance
    drop_substeps = True,
)

# Disable logging
# For large experiments, there is lots of logging. This can get messy...
logger = logging.getLogger()
logger.disabled = True

# Execute Experiment
raw_results = experiment.run()

time: 656 ms (started: 2023-03-18 18:48:07 +05:00)


### Indexing a Large-scale Experiment Dataset

In [26]:
# Create a Pandas DataFrame from the raw results
df = pd.DataFrame(experiment.results)
df

,stage,timestamp,eth_price,eth_supply,eth_staked,supply_inflation,network_issuance,pow_issuance,number_of_validators_in_activation_queue,average_effective_balance,...,total_network_costs,total_revenue,total_profit,total_revenue_yields,total_profit_yields,simulation,subset,run,substep,timestep
0,NaN,NaT,1251.477131,1.000000e+08,1.874915e+07,0.000000,0.000000,0,0,3.200000e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0,0,1,0,0
1,4.0,2023-03-18 18:47:55.127169,1251.477131,9.999857e+07,1.767632e+07,-0.005225,-1430.453509,0,0,3.200000e+10,...,2.133671e+05,2.535066e+06,2.321699e+06,0.041856,0.038333,0,0,1,15,1
2,4.0,2023-03-19 18:47:55.127169,1251.477131,9.999714e+07,1.769792e+07,-0.005217,-1428.242171,0,0,3.200000e+10,...,2.136100e+05,2.537833e+06,2.324223e+06,0.041851,0.038328,0,0,1,15,2
3,4.0,2023-03-20 18:47:55.127169,1251.477131,9.999571e+07,1.771952e+07,-0.005222,-1429.802282,0,0,3.200000e+10,...,2.135980e+05,2.535881e+06,2.322283e+06,0.041767,0.038249,0,0,1,15,3
4,4.0,2023-03-21 18:47:55.127169,1251.477131,9.999428e+07,1.774112e+07,-0.005214,-1427.595541,0,0,3.200000e+10,...,2.138406e+05,2.538643e+06,2.324802e+06,0.041762,0.038244,0,0,1,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,4.0,2023-03-18 18:47:55.127169,4181.280000,1.221199e+08,3.482132e+07,-0.002105,-253941.104917,0,0,3.200000e+10,...,2.778232e+08,4.140552e+09,3.862729e+09,0.028853,0.026917,2,0,398,15,1
2599,NaN,NaT,1251.477131,1.223739e+08,1.874915e+07,0.000000,0.000000,0,0,3.200000e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,2,0,399,0,0
2600,4.0,2023-03-18 18:47:55.127169,4181.280000,1.221337e+08,3.576674e+07,-0.001991,-240128.017418,0,0,3.199999e+10,...,2.824146e+08,4.198308e+09,3.915894e+09,0.028482,0.026566,2,0,399,15,1
2601,NaN,NaT,1251.477131,1.223739e+08,1.874915e+07,0.000000,0.000000,0,0,3.200000e+10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,2,0,400,0,0


time: 203 ms (started: 2023-03-18 18:48:08 +05:00)


In [27]:
# Select each Simulation dataset
df_0 = df[df.simulation == 0]
df_1 = df[df.simulation == 1]
df_2 = df[df.simulation == 2]

datasets = [df_0, df_1, df_2]

# Determine size of Simulation datasets
for index, data in enumerate(datasets):
    runs = len(data.run.unique())
    subsets = len(data.subset.unique())
    timesteps = len(data.timestep.unique())
    
    print(f"Simulation {index} has {runs} runs * {subsets} subsets * {timesteps} timesteps = {runs * subsets * timesteps} rows")

Simulation 0 has 1 runs * 2 subsets * 361 timesteps = 722 rows
Simulation 1 has 1 runs * 1 subsets * 1081 timesteps = 1081 rows
Simulation 2 has 400 runs * 1 subsets * 2 timesteps = 800 rows
time: 125 ms (started: 2023-03-18 18:48:08 +05:00)


In [28]:
# Indexing simulation 0, run 1 (indexed from one!), subset 1, timestep 1
df.query("simulation == 0 and run == 1 and subset == 1 and timestep == 1")

,stage,timestamp,eth_price,eth_supply,eth_staked,supply_inflation,network_issuance,pow_issuance,number_of_validators_in_activation_queue,average_effective_balance,...,total_network_costs,total_revenue,total_profit,total_revenue_yields,total_profit_yields,simulation,subset,run,substep,timestep
362,4.0,2023-03-18 18:47:55.127169,1251.477131,9.999766e+07,17676320.0,-0.008536,-2337.157101,0,0,3.200000e+10,...,152092.303353,1.400347e+06,1.248255e+06,0.023121,0.02061,0,1,1,15,1


time: 141 ms (started: 2023-03-18 18:48:08 +05:00)
